In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, RepeatedKFold,StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression 
#from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,roc_auc_score,roc_curve, precision_score,f1_score,auc,precision_recall_curve,classification_report

import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings                        # To ignore any warnings 
warnings.filterwarnings("ignore")
#from xgboost import XGBClassifier

In [5]:
TRdata = pd.read_csv('loan_train.csv')
TTdata = pd.read_csv('loan_test.csv')
ntrain=TRdata.shape[0]

In [6]:
TRdata.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
TRdata.shape,TTdata.shape

((614, 13), (367, 12))

In [8]:
label_col='Loan_Status'

In [9]:
# ট্রেনিং এবং টেস্ট সেটকে একসাথে করছে 
df_all= pd.concat([TRdata, TTdata], ignore_index=True)


In [10]:
df_all

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban,NaN
977,LP002975,Male,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban,NaN
978,LP002980,Male,No,0,Graduate,No,3250,1993.0,126.0,360.0,NaN,Semiurban,NaN
979,LP002986,Male,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural,NaN


In [11]:
# Loan Status কলাম ম্যাপিং করছে 
df_all[label_col]=df_all[label_col].map({'Y':1,'N':0})


In [12]:
df_all

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban,NaN
977,LP002975,Male,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban,NaN
978,LP002980,Male,No,0,Graduate,No,3250,1993.0,126.0,360.0,NaN,Semiurban,NaN
979,LP002986,Male,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural,NaN


In [13]:
TRdata.shape,TTdata.shape,df_all.shape

((614, 13), (367, 12), (981, 13))

In [14]:

indMarrid=df_all[df_all['Married'].isnull()].index


In [15]:
indMarrid

Int64Index([104, 228, 435], dtype='int64')

In [16]:
df_all.loc[indMarrid,'Married']=TRdata['Married'].mode()[0]


In [17]:
df_all.loc[indMarrid,'Married']

104    Yes
228    Yes
435    Yes
Name: Married, dtype: object

In [18]:
indMarrid

Int64Index([104, 228, 435], dtype='int64')

In [20]:
df_all['TotalIncome']=df_all[['ApplicantIncome','CoapplicantIncome']].sum(1)


In [22]:
df_all.shape

(981, 14)

In [23]:
df_all.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1.0,5849.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0.0,6091.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1.0,3000.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1.0,4941.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1.0,6000.0


In [24]:
df_all['TotalIncome/LoanAmount']=df_all[['ApplicantIncome','CoapplicantIncome']].sum(1)/df_all['LoanAmount']


In [25]:
df_all.shape

(981, 15)

In [26]:
df_all.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome,TotalIncome/LoanAmount
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1.0,5849.0,NaN
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0.0,6091.0,47.585938
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1.0,3000.0,45.454545
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1.0,4941.0,41.175000
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1.0,6000.0,42.553191


In [32]:
df_all['Loan_Amount_Term/LoanAmount']=df_all[['ApplicantIncome','CoapplicantIncome']].sum(1)/df_all['Loan_Amount_Term']

In [33]:
df_all.shape

(981, 16)

In [34]:
df_all.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome,TotalIncome/LoanAmount,Loan_Amount_Term/LoanAmount
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1.0,5849.0,NaN,16.247222
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0.0,6091.0,47.585938,16.919444
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1.0,3000.0,45.454545,8.333333
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1.0,4941.0,41.175000,13.725000
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1.0,6000.0,42.553191,16.666667


In [30]:
def BiningContiCols_over_yBinary(contiColList,dataframe,bins=4):
    """
        Pass a list of continues columns , It will convert it into four bins and 

    """

    group=[i for i in range(1,bins+1)]
    dict1={'Low':1,'Average':2,'High':3, 'Very high':4}
    binsdf= pd.DataFrame()
    for conti_colname in contiColList:
        _colData=pd.DataFrame({conti_colname: dataframe[conti_colname]
                              },index=dataframe.index)
        _n_cols=pd.qcut(dataframe[conti_colname],q=bins,duplicates='drop').nunique()
        _colData['bins']=pd.qcut(dataframe[conti_colname], q=bins,duplicates='drop',labels=group[:_n_cols])
        
        binsdf[conti_colname+'_bins'] = _colData['bins']#.map(dict1)
        binsdf=binsdf.astype('int')
    return binsdf

In [31]:
ContiBins=BiningContiCols_over_yBinary(['LoanAmount', 'CoapplicantIncome', 'ApplicantIncome','Loan_Amount_Term/LoanAmount'],df_all,5)
ContiBins=ContiBins.astype('object')
df_all[ContiBins.columns]=ContiBins

ValueError: Cannot convert float NaN to integer